In [1]:
from selenium.webdriver import Chrome
from selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException, TimeoutException,NoSuchFrameException, WebDriverException, UnexpectedAlertPresentException
from selenium.webdriver.common.keys import Keys
import time
from tqdm import tqdm

import urllib.request

import re

In [2]:
# path 설정
driver = Chrome('C:/Users/Administrator/chromedriver.exe')

## (1). 파페치

In [3]:
# 가방
url = 'https://www.farfetch.com/kr/shopping/women/bags-purses-1/items.aspx?view=90&category=136033|135999|136035|136315|136297|137188|137189|136311|137170'

In [3]:
# 원피스
url = 'https://www.farfetch.com/kr/shopping/women/clothing-1/items.aspx?page=1&view=90&category=135979'
url = 'https://www.farfetch.com/kr/shopping/women/clothing-1/items.aspx?page={input_num}&view=90&category=135979'.format(input_num = page_num)

In [4]:
driver.get(url)

In [5]:
## 페이지 넘버를 알기위해, 스크롤 다운
body = driver.find_element_by_css_selector('body')
for i in range(45):
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(0.3)

time.sleep(1)
# 페이지넘버
total_page = driver.find_element_by_class_name('_5fd441._7c0d7c._e7b42f')
total_page_num = int(total_page.text.split(' ')[-1])
total_page_num

263

In [3]:
## 이미지 링크와 각 이미지별 브랜드 저장하는 함수
def link_brand_img(url):
    driver.get(url)
    global img_links_list
    global img_brands_list
    
    for i in range(1000, 15000, 1000):
        driver.execute_script("window.scrollTo(0, {size})".format(size = i))
        time.sleep(0.5)
    time.sleep(1)
        
    img_links = driver.find_elements_by_css_selector('a > div._9c4bad._976d71 > img')
    img_links_list = [img_link.get_attribute('src') for img_link in img_links]
    
    img_brands = driver.find_elements_by_css_selector('h3._346238')
    img_brands_list = [brand_name.text for brand_name in img_brands]
    
    # 브랜드 - 특수문자 제거
    for i, brand in enumerate(img_brands_list):
        document = re.sub('[*?\|:<>/^%&$#@_]', '', brand)
        img_brands_list[i] = document

In [4]:
img_number = 9944 # 폴더에 저장된 맨 마지막 번호에다가 + 1 

for page_num in tqdm(range(105, 131)): # 중간에 오류나면 페이지 조정
    url = 'https://www.farfetch.com/kr/shopping/women/clothing-1/items.aspx?page={input_num}&view=90&category=135979'.format(input_num = page_num)
    link_brand_img(url)
    
    for i in range(0, len(img_links_list)):
        urllib.request.urlretrieve(img_links_list[i], 'dress/{number}_{brand_name}.jpg'.format(number = img_number, brand_name = img_brands_list[i]))
        time.sleep(0.1)
        img_number+=1

100%|███████████████████████████████████████████████████████████████████████████████| 26/26 [1:41:26<00:00, 233.49s/it]
